일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/SVC (C=0.1) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15106, 2770)
y shape = (15106,)
# features = 2770
# L words = 15106


In [2]:
def get_sample_r(li, topk=5):
    nonzero = x[li,:].nonzero()[1]
    base = min(50, len(nonzero)//2)
    return [vocabs[r] for r in sorted(nonzero, key=lambda x:word_frequency[0,x], reverse=True)[base:base+topk]]

word_frequency = x.sum(axis=0)
word_frequency

matrix([[  26,   32,   33, ...,   16, 1938,   95]], dtype=int64)

In [3]:
# sparse matrix type
print(classifier.dual_coef_.shape)

# sum = 0 ?? alpha sum은 1아닌가?
print(classifier.dual_coef_.data.sum())

(1, 5012)
0.0


In [4]:
# Support vectors
classifier.support_vectors_.shape

(5012, 2770)

In [5]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.33178869323447635

In [6]:
# Vector row id
classifier.support_

array([    3,     6,    26, ..., 15098, 15099, 15103], dtype=int32)

In [7]:
base = 0
topk = 40
for idx, l in sorted(enumerate(classifier.support_), key=lambda x:abs(classifier.dual_coef_[0,x[0]]), reverse=True)[base :base + topk]:
    print('%s%s - %s (%.3f)' % (x_words[l], '' if y[l] == 1 else '(다)', get_sample_r(l), classifier.dual_coef_[0,idx]))

맞서(다) - ['자', '거나', '려고', '기로', '려는'] (-0.100)
맡(다) - ['았기', '기에는', '으면서도', '으려', '았는데'] (-0.100)
굽어보(다) - ['다', '기도', '니', '고는', '곤'] (-0.100)
든(다) - ['다거나', '다면서', '다든가', '다고도', '다고들'] (-0.100)
밉(다) - ['지도', '든', '기만', '질', '기까지'] (-0.100)
따라다니(다) - ['다', '다가', '기도', '니', '기는'] (-0.100)
떠나(다) - ['긴', '고도', '질', '신', '자는'] (-0.100)
높이(다) - ['거나', '진', '지는', '지도', '려고'] (-0.100)
다닌(다) - ['다면서', '다고는', '다지만', '다던', '답시고'] (-0.100)
구(다) - ['느냐는', '웠던', '워진', '우면', '워지는'] (-0.100)
식(다) - ['었지만', '을까', '어가는', '었으며', '어버린'] (-0.100)
뚫어지(다) - ['면서', '도록'] (-0.100)
두드려(다) - ['주지', '주면', '대는', '주며', '보려고'] (-0.100)
다듬(다) - ['고자', '긴', '어져', '어지는', '는다고'] (-0.100)
휘두르(다) - ['자', '거나', '지도', '려고', '고는'] (-0.100)
되살려(다) - ['내는', '낼', '져야', '내고', '내어'] (-0.100)
맛있(다) - ['기로', '기는', '었는데', '다며', '잖아'] (-0.100)
걸맞(다) - ['지도', '는다는', '는다고', '게도', '았고'] (-0.100)
생겨났(다) - ['는가', '을까', '는지도', '을지', '는가를'] (-0.100)
샀(다) - ['거나', '는지', '더라도', '으나', '더니'] (-0.100)
깨지(다) - ['자', '거나', '진', '구', '지는'] (-0.10

In [8]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 2
number of support vectors = 5012

alpha= -0.100, count=2506 (0.500)
alpha= 0.100, count=2506 (0.500)


In [9]:
classifier.n_support_

array([2506, 2506], dtype=int32)

Support Vector Machine은 약 30%의 매우 높은 support vector를 모델에 저장하고 있으며, 각각 support vector의 가중치 역시 다르지 않다. negative는 모두 -0.1, positive는 모두 0.1 weight를 가지고 있다. k-NN classifier와 다르지 않을 정도의 support vector ratio이며, 질 좋은 support vector를 선택하지도 (equal alpha) 않는다. 

## Regularization을 약하게 하는 경우

In [10]:
with open('../models/Support Vector Machine (rbf, C=10.0) norml30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)
x = x.tocsr()

x shape = (15106, 2770)
y shape = (15106,)
# features = 2770
# L words = 15106


In [11]:
# Proportion of Support vector 
classifier.n_support_.sum() / x.shape[0]

0.20243611809876871

In [12]:
from collections import Counter
alpha_count = Counter(classifier.dual_coef_.data)
print('number of alphas = %d' % len(alpha_count))
print('number of support vectors = %d\n' % len(classifier.dual_coef_.data))

for alpha, count in sorted(alpha_count.items(), key=lambda x:(x[1], abs(x[0])), reverse=True)[:50]:
    print('alpha= %.3f, count=%d (%.3f)' % (alpha, count, count/classifier.n_support_.sum()))

number of alphas = 22
number of support vectors = 3058

alpha= -10.000, count=1521 (0.497)
alpha= 10.000, count=1517 (0.496)
alpha= -9.893, count=1 (0.000)
alpha= 8.672, count=1 (0.000)
alpha= 7.519, count=1 (0.000)
alpha= 6.975, count=1 (0.000)
alpha= -6.911, count=1 (0.000)
alpha= 6.827, count=1 (0.000)
alpha= 6.430, count=1 (0.000)
alpha= 5.897, count=1 (0.000)
alpha= 5.053, count=1 (0.000)
alpha= 4.676, count=1 (0.000)
alpha= -4.607, count=1 (0.000)
alpha= 4.067, count=1 (0.000)
alpha= -4.058, count=1 (0.000)
alpha= 3.924, count=1 (0.000)
alpha= -3.089, count=1 (0.000)
alpha= 2.861, count=1 (0.000)
alpha= 2.806, count=1 (0.000)
alpha= 1.297, count=1 (0.000)
alpha= 1.081, count=1 (0.000)
alpha= 0.474, count=1 (0.000)


Regularization 을 약하게 걸어도 대부분은 -10, 10의 weight를 가지며, support vector ratio가 20%로 줄어들었지만 여전히 많은 숫자이다. instance learning과 다르지 않다